In [154]:
# critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 
# 'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 
# 'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0},
# 'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0,'You, Me and Dupree': 2.5}, 
# 'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 
# 'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
# 'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

critics = {'Lisa Rose': {'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, }, 
'Mick LaSalle': {'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})

            result[item][person] = prefs[person][item]
    return result

filmCritics = transformPrefs(critics)


In [155]:
import datetime
print(type(critics['Lisa Rose']))

rates = []
films = []
users = []

for i in critics:
    for j in critics[i]:
        temp = {
            'critic': i,
            'film': j,
            'rate': critics[i][j],
            'createdAt': datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
        }
        rates.append(temp)
        if j not in films:
            films.append(j)
        if i not in users:
            users.append(i)

# rates   
    

<class 'dict'>


In [156]:
import math

def sim_pearson_dict(data, item1, item2):
    item1Rates = [d for d in data if d['film'] == item1]
    item2Rates = [d for d in data if d['film'] == item2]

    n = 0
    sum1 = 0
    sum2 = 0
    sum1Sq = 0
    sum2Sq = 0
    pSum = 0

    for i in item1Rates:
        for j in item2Rates:
            if i['critic'] == j['critic']:
                n += 1
                sum1 += i['rate']
                sum2 += j['rate']
                sum1Sq += pow(i['rate'], 2)
                sum2Sq += pow(j['rate'], 2)
                pSum += i['rate'] * j['rate']

    if n == 0:
        return 0

    num = pSum - (sum1 * sum2 / n)
    den = math.sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    
    if den == 0: 
        return 0

    r = num / den
    return r

In [157]:
def sim_distance(data, item1, item2):
    item1Rates = [d for d in data if d['film'] == item1]
    item2Rates = [d for d in data if d['film'] == item2]

    sum = 0

    for i in item1Rates:
        for j in item2Rates:
            if i['critic'] == j['critic']:
                sum += pow(i['rate'] - j['rate'], 2)
                
    return 1 / (1 + sum)

In [158]:
def topMatches(similarity = sim_distance):
    # print(prefs)
    # print(person)
    # scores = [(similarity(prefs, item, other), other['film']) for other in prefs if other['film'] != item]
    # scores = [(similarity(rates, item1, item2), item1, item2) for item1 in films for item2 in films if item1 != item2]

    scores = []
    result = {}

    for i in range(len(films)):
        temp = []
        # for j in range(i + 1, len(films)):
        for j in range(len(films)):
            if(i != j):
                temp.append((similarity(rates, films[i], films[j]), films[j]))

        temp.sort()
        temp.reverse()
        scores.append(temp)

        result[films[i]] = temp        
    return result

    scores.sort()
    scores.reverse()
    return scores

In [159]:
scores = topMatches()
# scores

In [160]:
def getRecommendedItems(data, itemMatch, user):
    userRatings = [d for d in data if d['critic'] == user]
    print(userRatings)
    scores = {}
    totalSim = {}

    for userRating in userRatings:
        for (similarity, item2) in itemMatch[userRating['film']]:
            if any(d['film'] == item2 for d in userRatings):
                continue

            scores.setdefault(item2, 0)
            scores[item2] += similarity * userRating['rate']

            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [168]:
getRecommendedItems(rates, scores, users[6])

[{'critic': 'Toby', 'film': 'Snakes on a Plane', 'rate': 4.5, 'createdAt': '23.02.2022 11:37:46'}, {'critic': 'Toby', 'film': 'You, Me and Dupree', 'rate': 1.0, 'createdAt': '23.02.2022 11:37:46'}, {'critic': 'Toby', 'film': 'Superman Returns', 'rate': 4.0, 'createdAt': '23.02.2022 11:37:46'}]


[(3.238805970149254, 'The Night Listener'),
 (3.021739130434783, 'Just My Luck'),
 (2.851063829787234, 'Lady in the Water')]

In [168]:
import math

def sim_pearson(prefs, p1, p2):
    si = {}
    for item in prefs[p1]: # всі чуваки, які оцінили фільм
        if item in prefs[p2]: # чи оцінив цей чувак інший фільм
            si[item] = 1

    n = len(si)
    # print(si)
    if n == 0: 
        return 0
    
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])

    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])

    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])

    num = pSum - (sum1 * sum2 / n)
    den = math.sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0: 
        return 0

    r = num / den
    return r


In [169]:
sim_pearson(filmCritics, 'Snakes on a Plane', 'You, Me and Dupree')

-0.6454972243679047

In [224]:
def topMatches(prefs, person, n=5, similarity = sim_pearson):
 scores = [(similarity(prefs, person, other), other) for other in prefs if other != person]
 
 scores.sort()
 scores.reverse()
 return scores  #[0:n]

In [225]:
scores=[topMatches(filmCritics, item, similarity = sim_pearson) for item in films]
# scores

In [226]:
def getRecommendations(prefs, person, similarity = sim_pearson):
    totals = {}
    simSums = {}
    for other in prefs:
        if other == person: 
            continue
        sim = similarity(prefs, person, other)

        if sim <= 0:
            continue

        for item in prefs[other]:    
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item] * sim
                simSums.setdefault(item, 0)
                simSums[item] += sim

    rankings = [(total / simSums[item], item) for item, total in totals.items()]

    rankings.sort()
    rankings.reverse()
    return rankings

In [227]:
getRecommendations(critics, 'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [228]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})

            result[item][person] = prefs[person][item]
    return result

In [229]:
def sim_distance(prefs,person1,person2):
    si = {}

    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1

    if len(si) == 0: 
        return 0

    sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])
    return 1 / (1+sum_of_squares)


In [230]:
def calculateSimilarItems(prefs, n = 10):
    result = {}
    
    itemPrefs = transformPrefs(prefs)
    c = 0
    for item in itemPrefs:
        c += 1
        if c % 100 == 0: 
            print("%d / %d" % (c, len(itemPrefs)))
        scores=topMatches(itemPrefs, item, n = n, similarity = sim_pearson) # sim_pearson
        result[item] = scores        
    return result


In [231]:
itemsim = calculateSimilarItems(critics)
itemsim

{'Lady in the Water': [(0.7637626158259785, 'Snakes on a Plane'),
  (0.4879500364742689, 'Superman Returns'),
  (0.3333333333333333, 'You, Me and Dupree'),
  (-0.6123724356957927, 'The Night Listener'),
  (-0.9449111825230676, 'Just My Luck')],
 'Snakes on a Plane': [(0.7637626158259785, 'Lady in the Water'),
  (0.11180339887498941, 'Superman Returns'),
  (-0.3333333333333333, 'Just My Luck'),
  (-0.5663521139548527, 'The Night Listener'),
  (-0.6454972243679047, 'You, Me and Dupree')],
 'Just My Luck': [(0.5555555555555556, 'The Night Listener'),
  (-0.3333333333333333, 'Snakes on a Plane'),
  (-0.42289003161103106, 'Superman Returns'),
  (-0.4856618642571827, 'You, Me and Dupree'),
  (-0.9449111825230676, 'Lady in the Water')],
 'Superman Returns': [(0.6579516949597695, 'You, Me and Dupree'),
  (0.4879500364742689, 'Lady in the Water'),
  (0.11180339887498941, 'Snakes on a Plane'),
  (-0.1798471947990544, 'The Night Listener'),
  (-0.42289003161103106, 'Just My Luck')],
 'You, Me and

In [232]:
def getRecommendedItems(prefs, itemMatch, user):
    print(prefs)
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    for (item, rating) in userRatings.items():
        for (similarity, item2) in itemMatch[item]:
            if item2 in userRatings: 
                continue

            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating

            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [233]:
getRecommendedItems(critics, itemsim, 'Toby')

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0}, 'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5}, 'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5}, 'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman 

[(3.610031066802183, 'Lady in the Water'),
 (3.5313950341859766, 'The Night Listener'),
 (2.960999860724268, 'Just My Luck')]

In [24]:
books = {'книга1': {'1':3.0, '3':1.0, '5': 4.0, '6':2.0, '9':5.0, '10': 2.0},
            'книга2': {'2':2.0, '3':5.0, '7':5.0, '8':2.0},
            'книга3': {'2':3.0, '4':4.0, '5':4.0, '6':1.0, '9': 3.0, '10': 1.0},
            'книга4': {'1':5.0, '3':5.0, '7':3.0, '8': 2.0},
            'книга5': {'1':2.0, '3':3.0, '5':1.0, '6':4.0, '9': 3.0, '10':3.0},
            'книга6': {'1':2.0, '2':1.0, '4':2.0, '8': 5.0, '10': 4.0},
            'книга7': {'2':2.0, '5':2.0, '6':3.0, '7': 3.0, '9':1.0},
            'книга8': {'2':3.0, '3':4.0, '4':5.0, '6':1.0, '7':2.0, '8':3.0, '9':4.0},
            'книга9': {'1':1.0, '3':5.0, '5':5.0, '8': 1.0, '10': 5.0},
            'книга10': {'2':2.0, '4':1.0, '5':4.0, '6':2.0, '7':5.0, '9': 3.0}}

In [25]:
import math

def koef_pearson(prefs, book1, book2):
    if book1 == book2:
        return 1

    si = []
    for item in prefs[book1]: # всі чуваки, які оцінили фільм
        if item in prefs[book2]: # чи оцінив цей чувак інший фільм
            si.append(item)

    n = len(si)
    if n == 0: 
        return 0
    
    avg1 = sum([prefs[book1][it] for it in si]) / n
    avg2 = sum([prefs[book2][it] for it in si]) / n

    num = 0 
    sq_sum1 = 0
    sq_sum2 = 0

    for it in si:
        num += (prefs[book1][it] - avg1) * (prefs[book2][it] - avg2)
        sq_sum1 += pow((prefs[book1][it] - avg1), 2)
        sq_sum2 += pow((prefs[book2][it] - avg2), 2)

    den = math.sqrt(sq_sum1 * sq_sum2)
    if den == 0: 
        return 0

    r = ((num / den) + 1) / 2
    # r = (num / den)
    return round(r, 3)


In [26]:
def topMatches(prefs, person, n = 20):
    scores = [(koef_pearson(prefs, person, other), other) for other in prefs]# if other != person]

    scores.sort()
    scores.reverse()
    return scores  #[0:n]

In [27]:
import pandas as pd

scores = []

for book in books:
    scores.append({other: koef_pearson(books, book, other) for other in books})
    # print(scores)

df = pd.DataFrame(scores, index = books.keys())
df

,книга1,книга2,книга3,книга4,книга5,книга6,книга7,книга8,книга9,книга10
книга1,1.000,0.000,0.926,0.000,0.281,0.000,0.009,0.639,0.371,0.827
книга2,0.000,1.000,0.000,0.878,0.000,0.000,1.000,0.500,1.000,1.000
книга3,0.926,0.000,1.000,0.000,0.081,0.107,0.176,0.985,0.000,0.590
книга4,0.000,0.878,0.000,1.000,0.000,0.000,0.000,0.827,0.750,0.000
книга5,0.281,0.000,0.081,0.000,1.000,1.000,0.664,0.000,0.587,0.009
книга6,0.000,0.000,0.107,0.000,1.000,1.000,0.000,0.361,0.594,0.000
книга7,0.009,1.000,0.176,0.000,0.664,0.000,1.000,0.028,0.000,0.592
книга8,0.639,0.500,0.985,0.827,0.000,0.361,0.028,1.000,1.000,0.291
книга9,0.371,1.000,0.000,0.750,0.587,0.594,0.000,1.000,1.000,0.000
книга10,0.827,1.000,0.590,0.000,0.009,0.000,0.592,0.291,0.000,1.000


In [28]:
scores=[topMatches(books, item) for item in books]
scores

[[(1, 'книга1'),
  (0.926, 'книга3'),
  (0.827, 'книга10'),
  (0.639, 'книга8'),
  (0.371, 'книга9'),
  (0.281, 'книга5'),
  (0.009, 'книга7'),
  (0.0, 'книга6'),
  (0, 'книга4'),
  (0, 'книга2')],
 [(1.0, 'книга9'),
  (1.0, 'книга7'),
  (1, 'книга2'),
  (1.0, 'книга10'),
  (0.878, 'книга4'),
  (0.5, 'книга8'),
  (0, 'книга6'),
  (0, 'книга5'),
  (0, 'книга3'),
  (0, 'книга1')],
 [(1, 'книга3'),
  (0.985, 'книга8'),
  (0.926, 'книга1'),
  (0.59, 'книга10'),
  (0.176, 'книга7'),
  (0.107, 'книга6'),
  (0.081, 'книга5'),
  (0, 'книга9'),
  (0, 'книга4'),
  (0, 'книга2')],
 [(1, 'книга4'),
  (0.878, 'книга2'),
  (0.827, 'книга8'),
  (0.75, 'книга9'),
  (0, 'книга7'),
  (0.0, 'книга6'),
  (0, 'книга5'),
  (0, 'книга3'),
  (0, 'книга10'),
  (0, 'книга1')],
 [(1.0, 'книга6'),
  (1, 'книга5'),
  (0.664, 'книга7'),
  (0.587, 'книга9'),
  (0.281, 'книга1'),
  (0.081, 'книга3'),
  (0.009, 'книга10'),
  (0.0, 'книга8'),
  (0, 'книга4'),
  (0, 'книга2')],
 [(1, 'книга6'),
  (1.0, 'книга5'),
  (0.5

In [29]:
# def getRecommendations(prefs, person):
#     totals = {}
#     simSums = {}
#     for other in prefs:
#         if other == person: 
#             continue
#         sim = koef_pearson(prefs, person, other)

#         if sim <= 0:
#             continue

#         for item in prefs[other]:    
#             if item not in prefs[person] or prefs[person][item] == 0:
#                 totals.setdefault(item, 0)
#                 totals[item] += prefs[other][item] * sim
#                 simSums.setdefault(item, 0)
#                 simSums[item] += sim

#     rankings = [(total / simSums[item], item) for item, total in totals.items()]

#     rankings.sort()
#     rankings.reverse()
#     return rankings

In [30]:
def calculateSimilarItems(prefs, n = 20):
    result = {}
    c = 0
    for item in prefs:
        c += 1
        if c % 100 == 0: 
            print("%d / %d" % (c, len(prefs)))

        scores=topMatches(prefs, item, n = n)
        result[item] = scores        
    return result


In [31]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})

            result[item][person] = prefs[person][item]
    return result

In [32]:
rates = transformPrefs(books)

In [33]:
def getRecommendedItems(prefs, itemMatch, user):
    userRatings = prefs[user]
    # print(userRatings)
    # print(itemMatch)
    scores = {}
    totalSim = {}

    for (item, rating) in userRatings.items():
        for (similarity, item2) in itemMatch[item]:
            if item2 in userRatings: 
                continue

            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating
            # print(scores)

            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    rankings=[(round(score/totalSim[item], 3),item) for item,score in scores.items( )]
    
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [34]:
similarity = calculateSimilarItems(books)
print(similarity)
recomendations = getRecommendedItems(rates, similarity, '8')
# print(rates['8'])
data = []
for item in recomendations:
    data.append([item[1], item[0]])

df = pd.DataFrame(data)
df

# recomendations

# similarity

{'книга1': [(1, 'книга1'), (0.926, 'книга3'), (0.827, 'книга10'), (0.639, 'книга8'), (0.371, 'книга9'), (0.281, 'книга5'), (0.009, 'книга7'), (0.0, 'книга6'), (0, 'книга4'), (0, 'книга2')], 'книга2': [(1.0, 'книга9'), (1.0, 'книга7'), (1, 'книга2'), (1.0, 'книга10'), (0.878, 'книга4'), (0.5, 'книга8'), (0, 'книга6'), (0, 'книга5'), (0, 'книга3'), (0, 'книга1')], 'книга3': [(1, 'книга3'), (0.985, 'книга8'), (0.926, 'книга1'), (0.59, 'книга10'), (0.176, 'книга7'), (0.107, 'книга6'), (0.081, 'книга5'), (0, 'книга9'), (0, 'книга4'), (0, 'книга2')], 'книга4': [(1, 'книга4'), (0.878, 'книга2'), (0.827, 'книга8'), (0.75, 'книга9'), (0, 'книга7'), (0.0, 'книга6'), (0, 'книга5'), (0, 'книга3'), (0, 'книга10'), (0, 'книга1')], 'книга5': [(1.0, 'книга6'), (1, 'книга5'), (0.664, 'книга7'), (0.587, 'книга9'), (0.281, 'книга1'), (0.081, 'книга3'), (0.009, 'книга10'), (0.0, 'книга8'), (0, 'книга4'), (0, 'книга2')], 'книга6': [(1, 'книга6'), (1.0, 'книга5'), (0.594, 'книга9'), (0.361, 'книга8'), (0.10

,0,1
0,книга5,3.520
1,книга3,3.196
2,книга1,2.265
3,книга10,2.225
4,книга7,2.027


In [35]:
scores=[topMatches(books, item) for item in books]
# scores

In [13]:
 # for item in reviews1:
    #     if any(review['userId'] == item['userId'] for review in reviews2):
    #         users.append(item['userId'])

# avg1 = 0
    # avg2 = 0

    # for user in users:
    #     for review1 in reviews1:
    #         if review1['userId'] == user:
    #             avg1 += review1['rate']

    #     for review2 in reviews2:
    #         if review2['userId'] == user:
    #             avg2 += review2['rate']

    # avg1 = avg1 / n
    # avg2 = avg2 / n